### Задание теме «Коллаборативная фильтрация»
Преподаватель: Алексей Кузьмин
ПАКЕТ SURPRISE

- используйте данные MovieLens 1M
- можно использовать любые модели из пакета
- получите RMSE на тестовом сете 0.87 и ниже

In [1]:
import pandas as pd
import numpy as np
from surprise import KNNWithMeans
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV

### 1. Качаем и загружаем оригинальный датасет
от сюда: [MovieLens 1M](https://grouplens.org/datasets/movielens/1m/)

In [2]:
movies = pd.read_table('./data/MovieLens 1M/movies.dat', sep='::', header=None, names=['movieId','title','genre'])
ratings = pd.read_table('./data/MovieLens 1M/ratings.dat', sep='::', header=None, names=['userId','movieId','rating', 'timestamp'])

/home/data/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/home/data/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [3]:
df = ratings.merge(movies.set_index('movieId'), on='movieId')[['userId', 'movieId', 'title', 'rating', 'timestamp']]

In [4]:
dataset = pd.DataFrame({
    'uid': df.userId,
    'iid': df.title,
    'rating': df.rating
})

In [5]:
reader = Reader(line_format='user item rating', rating_scale=(1,5))
df_prepared = Dataset.load_from_df(dataset, reader)

In [6]:
trainset, testset = train_test_split(df_prepared)

### 2. Используем SVD с оптимальными параметрами

In [7]:
algo = SVD(n_epochs=10, lr_all=0.005, reg_all=0.4)

In [8]:
algo.fit(trainset)

In [9]:
predicted = algo.test(testset)

In [10]:
accuracy.rmse(predicted, verbose=True)

RMSE: 0.9295


0.9295236884980543

### 3. Проверим, может быть kNN лучше

In [11]:
algo = KNNWithMeans(k=50, sim_options={'name': 'cosine', 'user_based': True})

In [12]:
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [13]:
predicted = algo.test(testset)

In [14]:
accuracy.rmse(predicted, verbose=True)

RMSE: 0.9363


0.9363153328405964

### Очевидно SVD лучше.
Получить оценку меньше 0.87 у меня не получилось т.к.:
- В оригинальном датасете рейтинги округлены до целого
- При попытке скормить датасет в GridSearch, моя машина "умирала"